## **Feature:** outliers

**Names:** Hailing Chen

### **What it does**
Automatically detects outliers and recommends optimal handling strategies based on your data's characteristics. Analyzes distribution patterns, sample sizes, and column types to suggest whether to cap, remove, transform, or fill outlier values. 

Provides context-aware recommendations like "Transform (highly skewed data)" or "Remove outliers (large dataset)" to help you make informed data cleaning decisions.

In [6]:
# Load dotenv
import os
from dotenv import load_dotenv
load_dotenv()

# Get API Key
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("OpenAI API Key not found")

# Import libraries
from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats
import io
import contextlib

# Langchain imports
from langchain_openai import ChatOpenAI  
from langchain.schema import HumanMessage, SystemMessage

### **Helper Functions**

- `analyze_outliers(df, iqr_multiplier=1.5, z_threshold=3, contamination=0.1)`: Analyze outliers and suggest handling strategies based on heuristics.
- `get_outlier_details(df, col, method='iqr', iqr_multiplier=1.5, z_threshold=3)`: Get detailed information about outliers in a specific column
- `handle_outliers(df, col, method='cap', percentiles=(5, 95), fill_value=None)`: Handle outliers in a specific column using methods including 'cap', 'remove', 'transform', 'fill'

In [1]:
def analyze_outliers(df, iqr_multiplier=1.5, z_threshold=3, contamination=0.1):
    """
    Analyze outliers and suggest handling strategies based on heuristics.
    
    Parameters:
    - df: pandas DataFrame
    - iqr_multiplier: multiplier for IQR method (default 1.5)
    - z_threshold: threshold for Z-score method (default 3)
    - contamination: expected proportion of outliers for isolation forest (default 0.1)
    
    Returns:
    - DataFrame with outlier analysis and suggestions for each numeric column
    """
    
    df.columns = df.columns.str.replace(' ', '')
    suggestions = {}
    
    # Only analyze numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    for col in numeric_cols:
        col_result = {}
        if df[col].isna().all():
            continue
            
        clean_data = df[col].dropna()
        col_result["dtype"] = str(df[col].dtype)
        col_result['sample_size'] = len(clean_data)
        # if len(clean_data) < 10:  # Skip if too few data points
        #     col_result['suggestion'] = "Insufficient data for outlier detection"
            # suggestions[col] = {
            #     "dtype": str(df[col].dtype),
            #     "sample_size": len(clean_data),
            #     "outliers_iqr": 0,
            #     "outliers_zscore": 0,
            #     "outlier_pct": 0,
            #     "suggestion": "Insufficient data for outlier detection"
            # }
            # continue
        
        try:
            # IQR Method
            Q1 = clean_data.quantile(0.25)
            Q3 = clean_data.quantile(0.75)
            IQR = Q3 - Q1
            
            if IQR == 0:  # No variance in data
                col_result["suggestion"] = "No variance in data - all values are the same"
                suggestions[col] = col_result
                continue
                
            lower_bound = Q1 - iqr_multiplier * IQR
            upper_bound = Q3 + iqr_multiplier * IQR
            outliers_iqr = ((clean_data < lower_bound) | (clean_data > upper_bound)).sum()
            col_result["outliers_iqr"] = outliers_iqr
        except Exception as e:
            col_result["outliers_iqr"] = 'error'
        
        try:
            # Z-Score Method
            z_scores = np.abs(stats.zscore(clean_data))
            outliers_zscore = (z_scores > z_threshold).sum()
            col_result["outliers_zscore"] = outliers_zscore
        except Exception as e:
            col_result["outliers_zscore"] = 'error'
        
        # Take the more conservative estimate
        outlier_count = min(col_result["outliers_iqr"], col_result["outliers_zscore"])
        col_result["outlier_pct"] = round(outlier_count / len(clean_data), 3)
        
        try:
            # Additional statistics for decision making
            n_unique = clean_data.nunique()
            col_result["skewness"] = round(clean_data.skew(), 3)
            col_result["kurtosis"] = round(clean_data.kurtosis(), 3)
            data_range = clean_data.max() - clean_data.min()
        except Exception as e:
            n_unique = len(clean_data.unique()) if len(clean_data) > 0 else 0
            col_result["skewness"] = 0.0
            col_result["kurtosis"] = 0.0
            data_range = 0
            print(f"Statistics calculation error for {col}: {e}")
        
                # Generate suggestions based on heuristics
        outlier_pct = col_result["outlier_pct"]
        skewness = col_result["skewness"]
        kurtosis = col_result["kurtosis"]
        
        try:
            # Special cases first (override general rules)
            col_lower = str(col).lower()

            if len(clean_data) < 10:  # Skip if too few data points
                suggestion = "Insufficient data for outlier detection"
            elif any(keyword in col_lower for keyword in ['id', 'index', 'key']):
                suggestion = "No action needed (ID/Index column)"
            elif 'age' in col_lower and (clean_data < 0).any():
                suggestion = "Cap to realistic range (negative age detected)"
            elif any(keyword in col_lower for keyword in ['price', 'cost', 'amount', 'salary']) and (clean_data < 0).any():
                suggestion = "Remove or cap (negative financial values)"
            elif any(keyword in col_lower for keyword in ['year', 'month', 'day']):
                suggestion = "Investigate outliers (date component)"
            elif any(keyword in col_lower for keyword in ['income', 'salary', 'revenue', 'population', 'sales']) and outlier_pct > 0.01:
                suggestion = "Transform (log transformation for multiplicative data)"
            
            # General rules based on outlier percentage
            elif outlier_pct == 0:
                suggestion = "No action needed (no outliers detected)"
            
            elif outlier_pct > 0.2:  # More than 20% outliers
                if n_unique < 20:  # Likely categorical numeric data
                    suggestion = "Investigate data type (possibly categorical codes)"
                else:
                    suggestion = "Investigate data quality (>20% outliers)"
            
            elif outlier_pct > 0.1:  # 10-20% outliers
                if abs(skewness) > 2:
                    suggestion = "Transform (highly skewed with many outliers)"
                elif col_result["sample_size"] > 1000:  # Large dataset
                    suggestion = "Remove outliers (large dataset can afford data loss)"
                else:
                    suggestion = "Cap outliers (preserve sample size)"
            
            elif outlier_pct > 0.05:  # 5-10% outliers
                if kurtosis > 3:  # Heavy-tailed distribution
                    suggestion = "Cap at percentiles (heavy-tailed distribution)"
                elif abs(skewness) > 1:
                    suggestion = "Transform or cap (skewed data)"
                elif col_result["sample_size"] < 100:  # Small dataset
                    suggestion = "Fill with median (small dataset)"
                else:
                    suggestion = "Cap at percentiles (moderate outliers)"
            
            elif outlier_pct > 0.01:  # 1-5% outliers
                if col_result["sample_size"] > 10000:  # Very large dataset
                    suggestion = "Remove outliers (large dataset)"
                elif n_unique < 50 and data_range > 1000:  # Likely measurement errors
                    suggestion = "Fill with median (likely measurement errors)"
                else:
                    suggestion = "Investigate individual cases (low outlier rate)"
            
            else:  # Less than 1% outliers
                suggestion = "No action needed (natural variation)"
            
            col_result["suggestion"] = suggestion
            
        except Exception as e:
            col_result["suggestion"] = f"Error generating suggestion: {str(e)}"
            print(f"Suggestion generation error for {col}: {e}")
        
        # Ensure all required keys are present with proper types
        suggestions[col] = col_result
        
    output_df = pd.DataFrame.from_dict(suggestions, orient="index")
    output_df.index.name = "variable"
    markdown_table = output_df.to_markdown()
    markdown_table += "\n\n---\n"
    return print(markdown_table)

    # return pd.DataFrame.from_dict(suggestions, orient="index")

In [3]:
def handle_outliers(df, col, method='cap', percentiles=(5, 95), fill_value=None):
    """
    Handle outliers in a specific column using various methods.
    
    Parameters:
    - df: pandas DataFrame
    - col: column name to process
    - method: 'cap', 'remove', 'transform', 'fill'
    - percentiles: tuple for capping (lower, upper percentiles)
    - fill_value: value to fill outliers with (for 'fill' method)
    
    Returns:
    - Modified DataFrame
    """
    if col not in df.columns:
        print(f"Column '{col}' not found")
        return df
    
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"Column '{col}' is not numeric")
        return df
    
    df_copy = df.copy()
    
    if method == 'cap':
        lower_cap = df_copy[col].quantile(percentiles[0] / 100)
        upper_cap = df_copy[col].quantile(percentiles[1] / 100)
        df_copy[col] = df_copy[col].clip(lower=lower_cap, upper=upper_cap)
        print(f"Capped outliers in '{col}' to [{lower_cap:.2f}, {upper_cap:.2f}]")
    
    elif method == 'remove':
        Q1 = df_copy[col].quantile(0.25)
        Q3 = df_copy[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        before_count = len(df_copy)
        df_copy = df_copy[(df_copy[col] >= lower_bound) & (df_copy[col] <= upper_bound)]
        after_count = len(df_copy)
        print(f"Removed {before_count - after_count} outlier rows from '{col}'")
    
    elif method == 'transform':
        # Log transformation for positive skewed data
        if (df_copy[col] > 0).all():
            df_copy[col] = np.log1p(df_copy[col])
            print(f"Applied log transformation to '{col}'")
        else:
            print(f"Cannot apply log transformation to '{col}' (contains non-positive values)")
            return df
    
    elif method == 'fill':
        if fill_value is None:
            fill_value = df_copy[col].median()
        
        Q1 = df_copy[col].quantile(0.25)
        Q3 = df_copy[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outlier_mask = (df_copy[col] < lower_bound) | (df_copy[col] > upper_bound)
        outlier_count = outlier_mask.sum()
        df_copy.loc[outlier_mask, col] = fill_value
        print(f"Filled {outlier_count} outliers in '{col}' with {fill_value}")
    
    else:
        print(f"Unknown strategy '{method}', outliers not handled.")
    
    return df_copy

In [2]:
helper_docs = """ Helper functions available:
- analyze_outliers(df, iqr_multiplier=1.5, z_threshold=3, contamination=0.1): Analyze outliers and suggest handling strategies based on heuristics.
- handle_outliers(df, col, method, percentiles=(5, 95), fill_value=None): Handle outliers in a specific column using methods defined.
    - supported methods: 'cap', 'remove', 'transform', 'fill'
    - 'cap' limits extreme values to specified percentiles without removing data points.
    - 'remove' removes outliers outside of IQR.
    - 'transform' applies log transformation to the specific column
    - 'fill' replaces outlier values with a specified value (default: median) or a custom value.

EXAMPLES:
- Print(analyze_outliers(df))
- User: apply log transformation to the column named 'a', Generatedf: df['a'] = handle_outliers(df, 'a', method='transform')
"""

In [4]:
helpers = {
    "analyze_outliers": analyze_outliers,
    "handle_outliers": handle_outliers,
    # add more here
}

# **MAIN FEATURE FUNCTION**

In [ ]:
def outliers(df, message):
    """
    Main function that gets called by the main router.
    MUST take (df, message) and return df
    """
    # suggestions = analyze_outliers(df)
    
    # Create message chain
    messages = []
    messages.append(SystemMessage(content=helper_docs))
    messages.append(SystemMessage(content=f"""
    You are a data cleaning agent trying to handle outliers.
    
    Dataset info: Shape: {df.shape}, Sample: {df.head(3).to_string()}

    Libraries available:
    - pd (pandas), np (numpy)
    - math, re, datetime
    - states from spicy
    
    Rules:
    - use analyze_outlier to provide information on outliers to user
    - Return only executable Python code, no explanations, no markdown blocks
    - Use helper functions if needed
    - ASSUME "df" IS ALREADY DEFINED
    - In order to generate a response/message to the user use PRINT STATEMENTS: print("message")
    - Write a detailed print message to summarise actions taken and reasons, such as suggestions and potential actions 
    """))
    messages.append(HumanMessage(content=f"User request: {message}"))
    
    # Call LLM with message chain
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    response = llm.invoke(messages)
    generated_code = response.content.strip()
    
    buffer = io.StringIO()
    try:
        exec_globals = {"df": df, 
                        "pd": pd, 
                        "np": np,
                        "stats": stats,
                        **helpers}
        with contextlib.redirect_stdout(buffer):
            exec(generated_code, exec_globals)
        output = buffer.getvalue().strip()
        
        if not output:
            output = "Code executed, but nothing was printed."
        return output
    
    except Exception as e:
        return f"Error running code:\n{e}\n\nGenerated code:\n{generated_code}"

# **Testing**

In [14]:
# # Enter CSV filename from "datasets" folder
# dataset_name = "sample-data.csv"

# # Build CSV path (to avoid import errors)
# load_dotenv()
# PROJECT_ROOT = Path(os.environ["PROJECT_ROOT"])
# path = PROJECT_ROOT / "datasets" / dataset_name

# df = pd.read_csv(path)
# test_df = df.copy()